# Extraction of Outcomes Hospitalization Rate & Ejection Fraction

This Notebook is using the timeseries data of hospitalization and the ejection fraction. For the Hospotatilzation the days a patient spend in a hopsital are summarized and for the ejection fraction the average value of a patient is identified. 

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import math

# Days in Hospital per Patient

In [ ]:
#Reading the hospitalization Events
hospitalization = pq.read_table('Cohort/Time_Series/Inpatient_Events_raw.parquet').to_pandas()

In [ ]:
hospitalization

In [ ]:
#load the dataframe with the final cohort size
mrn_without_lab=pq.read_table('Cohort/Feature_Extraction/Unsupervised_ALL_HF/LabValue_after_onset_HF_ALL_mmm_0_8_missing_values').to_pandas()

In [ ]:
hospitalization=hospitalization.sort_values(by=['medical_record_number','age_in_days'])

In [ ]:
mrn_list=mrn_without_lab['medical_record_number'].to_list()
len(mrn_list)

In [ ]:

hospitalization = hospitalization[~hospitalization['medical_record_number'].isin(mrn_list)]
hospitalization

In [ ]:
#loading dataframe with Onset Data: 
medium_data_set=pq.read_table('Cohort/Feature_Extraction/ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab.parquet').to_pandas()
medium_data_set=medium_data_set['HF_Onset_age_in_days']
medium_data_set

In [ ]:
hospitalization=pd.merge(hospitalization,medium_data_set,left_on='medical_record_number', right_index=True)

In [ ]:
hospitalization=hospitalization.loc[hospitalization['age_in_days']>=hospitalization['HF_Onset_age_in_days']]

In [ ]:
hospitalization

In [ ]:
#get count 
hospitalization=hospitalization.groupby('medical_record_number').count()
hospitalization=hospitalization[ ['age_in_days']]
hospitalization=hospitalization.rename(columns={'age_in_days': 'days_in_hospital'})

In [ ]:
hospitalization

In [ ]:
#save dataframe 
hospitalization.to_parquet('Cohort/Feature_Extraction/days_in_hospital.parquet')

# generate average for Ejection Fractions

In [ ]:
ef = pq.read_table('Cohort/Time_Series/Ejection_Fraction_raw.parquet').to_pandas()

In [ ]:
ef

In [ ]:
#load the dataframe with the final cohort size
mrn_without_lab=pq.read_table('Cohort/Feature_Extraction/Unsupervised_ALL_HF/LabValue_after_onset_HF_ALL_mmm_0_8_missing_values').to_pandas()

In [ ]:
mrn_list=mrn_without_lab['medical_record_number'].to_list()
len(mrn_list)

In [ ]:

ef = ef[~ef['medical_record_number'].isin(mrn_list)]
ef

In [ ]:
col=['medical_record_number','avg_ef']
result_ef=pd.DataFrame(columns=col)
result_ef

In [ ]:
for mrn in ef['medical_record_number'].unique(): 
    patient=ef.loc[ef['medical_record_number']==mrn]
    #get average EF: 
    av_ef=patient['numeric_value'].mean()
    result_ef=result_ef.append({'medical_record_number':mrn,'avg_ef':av_ef},ignore_index=True)

In [ ]:
patient

In [ ]:
av_ef

In [ ]:
result_ef

In [ ]:
#save dataframe 
result_ef.to_parquet('Cohort/Feature_Extraction/avg_EF.parquet')